<a href="https://colab.research.google.com/github/adityamishra5050/Multilabel-Product-Text-Reviews-Classification-/blob/main/BERT_Multilabel_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/bodywash-train.xlsx')

In [ ]:
final_test_df = pd.read_excel('/content/drive/MyDrive/bodywash-test.xlsx')

In [ ]:
df_encoded = pd.get_dummies(df, columns=['Notes'], prefix='Notes')
df_final = df_encoded.groupby('Core Item').max().reset_index()
df_final = df_final.fillna(0)


In [ ]:
df_final.head(11)

,Core Item,Notes_Accessibility,Notes_Brand Accountability,Notes_Brand For Me,Notes_Brand Value,Notes_Cleansing,Notes_Companion Approval,Notes_Convenience,Notes_Efficacy,Notes_Feel / Finish,Notes_Fragrance,Notes_Packaging,Notes_Price,Notes_Product Safety,Notes_Product Texture,Notes_Skin Care,Notes_Skin Texture Improvement
0,31069 we've looked every where for your body ...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,I love all the scents. I buy all three at onc...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,I see several in this pic I haven't tried! Go...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Iâ€™m a big fan of everything honestly. I use...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,The Best! Hands down,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,"""#UncommonScents""",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
6,"""2 Things: 1, PLEASE PLEASE PLEASE keep making...",1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
7,"""@Cremo Company i used the blue cedar and cypr...",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
8,"""@Cremo Company the bodywash and cologne work ...",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
9,"""@Swift Media Co. Cremo's bodywash,spray bottl...",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [ ]:
!sudo apt-get update
!sudo apt-get install nvidia-driver

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 1s (224 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package nvidia-driver


In [ ]:
!nvidia-smi

Thu Dec 14 06:09:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    28W /  70W |   4585MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys

In [ ]:
df_final.columns

Index(['Core Item', 'Notes_Accessibility', 'Notes_Brand Accountability',
       'Notes_Brand For Me', 'Notes_Brand Value', 'Notes_Cleansing',
       'Notes_Companion Approval', 'Notes_Convenience', 'Notes_Efficacy',
       'Notes_Feel / Finish', 'Notes_Fragrance', 'Notes_Packaging',
       'Notes_Price', 'Notes_Product Safety', 'Notes_Product Texture',
       'Notes_Skin Care', 'Notes_Skin Texture Improvement'],
      dtype='object')

In [ ]:
target_list = ['Notes_Accessibility', 'Notes_Brand Accountability',
       'Notes_Brand For Me', 'Notes_Brand Value', 'Notes_Cleansing',
       'Notes_Companion Approval', 'Notes_Convenience', 'Notes_Efficacy',
       'Notes_Feel / Finish', 'Notes_Fragrance', 'Notes_Packaging',
       'Notes_Price', 'Notes_Product Safety', 'Notes_Product Texture',
       'Notes_Skin Care', 'Notes_Skin Texture Improvement']

In [ ]:
# hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 2
LEARNING_RATE = 1e-05

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['Core Item']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [ ]:
train_size = 0.8
train_df1 = df_final.sample(frac=train_size, random_state=200).reset_index(drop=True)
test_df = df_final.drop(train_df1.index).reset_index(drop=True)

train_df = train_df1.sample(frac=train_size, random_state=200).reset_index(drop=True)
val_df = train_df1.drop(train_df.index).reset_index(drop=True)


In [ ]:
class CustomDataset1(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['Core Item']
        #self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            #'targets': torch.FloatTensor(self.targets[index])
        }

In [ ]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)
test_dataset = CustomDataset(test_df, tokenizer, MAX_LEN)

In [ ]:
final_test_dataset = CustomDataset1(final_test_df, tokenizer, MAX_LEN)

In [ ]:

train_data_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

test_data_loader = torch.utils.data.DataLoader(test_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [ ]:
final_test_dataset_loader = torch.utils.data.DataLoader(final_test_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 16)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
val_targets=[]
val_outputs=[]

In [ ]:
import torch
import numpy as np

def multilabel_accuracy(predicted, targets):
    correct_per_label = (predicted == targets.byte()).sum(dim=0)
    correct_samples = (correct_per_label == targets.size(0)).sum().item()
    total_samples = targets.size(0)
    accuracy = correct_samples / total_samples
    return accuracy

def train_model(n_epochs, training_loader, validation_loader, model, optimizer, checkpoint_path, best_model_path):
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf

    for epoch in range(1, n_epochs+1):
        train_loss = 0
        valid_loss = 0
        correct_train = 0
        total_train = 0
        correct_valid = 0
        total_valid = 0

        model.train()
        print('############# Epoch {}: Training Start   #############'.format(epoch))
        for batch_idx, data in enumerate(training_loader):
            ids = data['input_ids'].to(device, dtype=torch.long)
            mask = data['attention_mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.float)

            outputs = model(ids, mask, token_type_ids)

            optimizer.zero_grad()
            loss = loss_fn(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))

            # Calculate training accuracy
            predicted = torch.sigmoid(outputs) >= 0.5
            correct_train += multilabel_accuracy(predicted, targets)
            total_train += targets.size(0)

        train_accuracy = correct_train / total_train
        #print('Training Accuracy: {:.4f}'.format(train_accuracy))

        print('############# Epoch {}: Training End     #############'.format(epoch))

        print('############# Epoch {}: Validation Start   #############'.format(epoch))
        model.eval()
        with torch.no_grad():
            for batch_idx, data in enumerate(validation_loader, 0):
                ids = data['input_ids'].to(device, dtype=torch.long)
                mask = data['attention_mask'].to(device, dtype=torch.long)
                token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
                targets = data['targets'].to(device, dtype=torch.float)

                outputs = model(ids, mask, token_type_ids)

                loss = loss_fn(outputs, targets)
                valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))

                # Calculate validation accuracy
                predicted = torch.sigmoid(outputs) >= 0.5
                correct_valid += multilabel_accuracy(predicted, targets)
                total_valid += targets.size(0)

            valid_accuracy = correct_valid / total_valid
            #print('Validation Accuracy: {:.4f}'.format(valid_accuracy))

        print('############# Epoch {}: Validation End     #############'.format(epoch))

        # calculate average losses
        train_loss = train_loss / len(training_loader)
        valid_loss = valid_loss / len(validation_loader)

        # print training/validation statistics
        print('Epoch: {} \tAverage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
        ))

        # create checkpoint variable and add important data
        checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }

        # save checkpoint
        save_ckp(checkpoint, False, checkpoint_path, best_model_path)

        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,
                                                                                              valid_loss))
            # save checkpoint as the best model
            save_ckp(checkpoint, True, checkpoint_path, best_model_path)
            valid_loss_min = valid_loss

        print('############# Epoch {}  Done   #############\n'.format(epoch))

    return model


In [ ]:
from sklearn import metrics

In [ ]:
'''import torch
import numpy as np



def train_model(n_epochs, training_loader, validation_loader, model, optimizer, checkpoint_path, best_model_path):
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf

    for epoch in range(1, n_epochs+1):
        train_loss = 0
        valid_loss = 0
        correct_train = 0
        total_train = 0
        correct_valid = 0
        total_valid = 0

        model.train()
        print('############# Epoch {}: Training Start   #############'.format(epoch))
        for batch_idx, data in enumerate(training_loader):
            ids = data['input_ids'].to(device, dtype=torch.long)
            mask = data['attention_mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.float)

            outputs = model(ids, mask, token_type_ids)

            optimizer.zero_grad()
            loss = loss_fn(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))

            # Calculate training accuracy
            predicted = torch.sigmoid(outputs).cpu().detach().numpy() >= 0.5
            correct_train += metrics.accuracy_score(predicted, targets)
            total_train += targets.size(0)

        train_accuracy = correct_train / total_train
        print('Training Accuracy: {:.4f}'.format(train_accuracy))

        print('############# Epoch {}: Training End     #############'.format(epoch))

        print('############# Epoch {}: Validation Start   #############'.format(epoch))
        model.eval()
        with torch.no_grad():
            for batch_idx, data in enumerate(validation_loader, 0):
                ids = data['input_ids'].to(device, dtype=torch.long)
                mask = data['attention_mask'].to(device, dtype=torch.long)
                token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
                targets = data['targets'].to(device, dtype=torch.float)

                outputs = model(ids, mask, token_type_ids)

                loss = loss_fn(outputs, targets)
                valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))

                # Calculate validation accuracy
                predicted = torch.sigmoid(outputs).cpu().detach().numpy() >= 0.5
                correct_valid += metrics.accuracy_score(predicted, targets)
                total_valid += targets.size(0)

            valid_accuracy = correct_valid / total_valid
            print('Validation Accuracy: {:.4f}'.format(valid_accuracy))

        print('############# Epoch {}: Validation End     #############'.format(epoch))

        # calculate average losses
        train_loss = train_loss / len(training_loader)
        valid_loss = valid_loss / len(validation_loader)

        # print training/validation statistics
        print('Epoch: {} \tAverage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
        ))

        # create checkpoint variable and add important data
        checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }

        # save checkpoint
        save_ckp(checkpoint, False, checkpoint_path, best_model_path)

        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,
                                                                                              valid_loss))
            # save checkpoint as the best model
            save_ckp(checkpoint, True, checkpoint_path, best_model_path)
            valid_loss_min = valid_loss

        print('############# Epoch {}  Done   #############\n'.format(epoch))

    return model'''


"import torch\nimport numpy as np\n\n\n\ndef train_model(n_epochs, training_loader, validation_loader, model, optimizer, checkpoint_path, best_model_path):\n    # initialize tracker for minimum validation loss\n    valid_loss_min = np.Inf\n\n    for epoch in range(1, n_epochs+1):\n        train_loss = 0\n        valid_loss = 0\n        correct_train = 0\n        total_train = 0\n        correct_valid = 0\n        total_valid = 0\n\n        model.train()\n        print('############# Epoch {}: Training Start   #############'.format(epoch))\n        for batch_idx, data in enumerate(training_loader):\n            ids = data['input_ids'].to(device, dtype=torch.long)\n            mask = data['attention_mask'].to(device, dtype=torch.long)\n            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)\n            targets = data['targets'].to(device, dtype=torch.float)\n\n            outputs = model(ids, mask, token_type_ids)\n\n            optimizer.zero_grad()\n          

In [ ]:
'''def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path):

  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf


  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        #if batch_idx%5000==0:
         #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)

    print('############# Epoch {}: Training End     #############'.format(epoch))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################
    # validate the model #
    ######################

    model.eval()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }

        # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)

      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model'''

"def train_model(n_epochs, training_loader, validation_loader, model,\n                optimizer, checkpoint_path, best_model_path):\n\n  # initialize tracker for minimum validation loss\n  valid_loss_min = np.Inf\n\n\n  for epoch in range(1, n_epochs+1):\n    train_loss = 0\n    valid_loss = 0\n\n    model.train()\n    print('############# Epoch {}: Training Start   #############'.format(epoch))\n    for batch_idx, data in enumerate(training_loader):\n        #print('yyy epoch', batch_idx)\n        ids = data['input_ids'].to(device, dtype = torch.long)\n        mask = data['attention_mask'].to(device, dtype = torch.long)\n        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)\n        targets = data['targets'].to(device, dtype = torch.float)\n\n        outputs = model(ids, mask, token_type_ids)\n\n        optimizer.zero_grad()\n        loss = loss_fn(outputs, targets)\n        #if batch_idx%5000==0:\n         #   print(f'Epoch: {epoch}, Training Loss:  {loss.it

In [ ]:
import os

# Define the directory path
directory_path = '/content/drive/MyDrive/datasets/multi-label'

# Create the directory if it doesn't exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)


In [ ]:
ckpt_path = "/content/drive/MyDrive/datasets/multi-label/curr_ckpt"
best_model_path = "/content/drive/MyDrive/datasets/multi-label/best_model.pt"

In [ ]:
trained_model = train_model(25, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)


############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Average Training Loss: 0.001402 	Average Validation Loss: 0.004460
Validation loss decreased (inf --> 0.004460).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Average Training Loss: 0.001042 	Average Validation Loss: 0.003724
Validation loss decreased (0.004460 --> 0.003724).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epoch

In [ ]:
def test_model(test_loader, model, device):
    model.eval()
    all_targets = []
    all_outputs = []

    with torch.no_grad():
        for data in test_loader:
            ids = data['input_ids'].to(device, dtype=torch.long)
            mask = data['attention_mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.float)

            outputs = model(ids, mask, token_type_ids)

            all_targets.extend(targets.cpu().detach().numpy().tolist())
            all_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    return all_targets, all_outputs

# Test the model on the test set
test_targets, test_outputs = test_model(test_data_loader, trained_model, device)

# Convert the lists to NumPy arrays for further evaluation
test_targets = np.array(test_targets)
test_outputs = np.array(test_outputs)

# Evaluate metrics (accuracy, hamming loss, precision, recall, f1 score)
from sklearn.metrics import accuracy_score, hamming_loss, precision_recall_fscore_support

# Binary thresholding for multi-label classification
threshold = 0.4
test_preds = (test_outputs > threshold).astype(int)

# Calculate metrics
accuracy = accuracy_score(test_targets, test_preds)
hamming_loss_value = hamming_loss(test_targets, test_preds)
precision, recall, f1, _ = precision_recall_fscore_support(test_targets, test_preds, average='weighted')

# Print metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Hamming Loss: {hamming_loss_value:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.8258
Hamming Loss: 0.0165
Precision: 0.9371
Recall: 0.9386
F1 Score: 0.9370


In [ ]:
import pandas as pd

# Convert the binary predictions to labels
def binary_to_labels(binary_predictions, target_list):
    labels = []
    for row in binary_predictions:
        label_indices = [i for i, value in enumerate(row) if value == 1]
        labels.append([target_list[idx] for idx in label_indices])
    return labels

# Convert the binary predictions to labels for test data
test_pred_labels = binary_to_labels(test_preds, target_list)

# Add a new column to the test dataframe with the predicted labels
test_df['Predicted_Labels'] = test_pred_labels

# Display the test dataframe with the predicted labels
print(test_df[['Core Item', 'Predicted_Labels']])


                                             Core Item  \
0            Smells great-My husband loves this one.\n   
1    Smells great-My husband only likes this scent ...   
2    Smells great-My husband really enjoyed this. H...   
3    Smells great-My husband tried this and loved t...   
4    Smells great-Nice and thick gel body wash. Nic...   
..                                                 ...   
707  â€œQuality productâ€-I've been ordering from ...   
708  â€œSent a body wash with a broken lid that had...   
709  â€œVery smooth shave--just ask my wifeâ€-I ha...   
710  â€œWorth every cent paid!â€-Have been orderin...   
711  â€œgreat productsâ€-I have never been as sati...   

                                      Predicted_Labels  
0    [Notes_Brand Value, Notes_Companion Approval, ...  
1          [Notes_Companion Approval, Notes_Fragrance]  
2                 [Notes_Convenience, Notes_Fragrance]  
3    [Notes_Brand Value, Notes_Companion Approval, ...  
4             [Not

In [ ]:
test_df.head()

,Core Item,Notes_Accessibility,Notes_Brand Accountability,Notes_Brand For Me,Notes_Brand Value,Notes_Cleansing,Notes_Companion Approval,Notes_Convenience,Notes_Efficacy,Notes_Feel / Finish,Notes_Fragrance,Notes_Packaging,Notes_Price,Notes_Product Safety,Notes_Product Texture,Notes_Skin Care,Notes_Skin Texture Improvement,Predicted_Labels
0,Smells great-My husband loves this one.\n,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,"[Notes_Brand Value, Notes_Companion Approval, ..."
1,Smells great-My husband only likes this scent ...,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,"[Notes_Companion Approval, Notes_Fragrance]"
2,Smells great-My husband really enjoyed this. H...,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,"[Notes_Convenience, Notes_Fragrance]"
3,Smells great-My husband tried this and loved t...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,"[Notes_Brand Value, Notes_Companion Approval, ..."
4,Smells great-Nice and thick gel body wash. Nic...,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,"[Notes_Fragrance, Notes_Product Texture]"


In [ ]:
def final_test_model(final_test_dataset_loader, model, device):
    model.eval()
    #all_targets = []
    all_outputs = []

    with torch.no_grad():
        for data in final_test_dataset_loader:
            ids = data['input_ids'].to(device, dtype=torch.long)
            mask = data['attention_mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            #targets = data['targets'].to(device, dtype=torch.float)

            outputs = model(ids, mask, token_type_ids)

            #all_targets.extend(targets.cpu().detach().numpy().tolist())
            all_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    return all_outputs


final_test_outputs = final_test_model(final_test_dataset_loader, trained_model, device)

# Convert the lists to NumPy arrays for further evaluation
#test_targets = np.array(test_targets)
final_test_outputs = np.array(final_test_outputs)



# Binary thresholding for multi-label classification
threshold = 0.5
final_test_preds = (final_test_outputs > threshold).astype(int)

In [ ]:
# Convert the binary predictions to labels for test data
final_test_pred_labels = binary_to_labels(final_test_preds, target_list)

# Add a new column to the test dataframe with the predicted labels
final_test_df['Predicted_Labels'] = final_test_pred_labels

# Display the test dataframe with the predicted labels
print(final_test_df[['Core Item', 'Predicted_Labels']])

                                             Core Item  \
0    Sons favorite scent-Can't buy in store so my s...   
1    My Date Loved its Scent-This body wash will ma...   
2    Great product, been using for years. - I look ...   
3    Works better than advertised.-This product wor...   
4    Great body wash!!-This body wash is great. It ...   
..                                                 ...   
196  Wonderful stuff, but what's with this price???...   
197  Smells great!-I got this body wash a couple we...   
198  Love the new body wash scents-Great scent. Lea...   
199  Love it-I've been using Old Spice body wash an...   
200  This stuff rocks-Harryâ€™s fig has a bright, f...   

                                      Predicted_Labels  
0               [Notes_Accessibility, Notes_Fragrance]  
1    [Notes_Brand Value, Notes_Companion Approval, ...  
2                 [Notes_Brand Value, Notes_Fragrance]  
3    [Notes_Feel / Finish, Notes_Skin Texture Impro...  
4    [Notes_Cleans

In [ ]:
final_test_df.head()

,Core Item,Notes,Predicted_Labels
0,Sons favorite scent-Can't buy in store so my s...,NaN,"[Notes_Accessibility, Notes_Fragrance]"
1,My Date Loved its Scent-This body wash will ma...,NaN,"[Notes_Brand Value, Notes_Companion Approval, ..."
2,"Great product, been using for years. - I look ...",NaN,"[Notes_Brand Value, Notes_Fragrance]"
3,Works better than advertised.-This product wor...,NaN,"[Notes_Feel / Finish, Notes_Skin Texture Impro..."
4,Great body wash!!-This body wash is great. It ...,NaN,"[Notes_Cleansing, Notes_Fragrance, Notes_Produ..."


In [ ]:
final_test_df1 = final_test_df.drop(columns = ['Notes'])

In [ ]:
final_test_df1.head(20)

,Core Item,Predicted_Labels
0,Sons favorite scent-Can't buy in store so my s...,"[Notes_Accessibility, Notes_Fragrance]"
1,My Date Loved its Scent-This body wash will ma...,"[Notes_Brand Value, Notes_Companion Approval, ..."
2,"Great product, been using for years. - I look ...","[Notes_Brand Value, Notes_Fragrance]"
3,Works better than advertised.-This product wor...,"[Notes_Feel / Finish, Notes_Skin Texture Impro..."
4,Great body wash!!-This body wash is great. It ...,"[Notes_Cleansing, Notes_Fragrance, Notes_Produ..."
5,tell me why I started washing my face with old...,[]
6,This is a great product! Smells amazing and ha...,"[Notes_Cleansing, Notes_Feel / Finish, Notes_F..."
7,Clearer skin-Nice texture and aroma that seems...,"[Notes_Fragrance, Notes_Product Texture]"
8,Charcoal is good mineral for scrub the skin so...,"[Notes_Cleansing, Notes_Fragrance, Notes_Produ..."
9,"""Nivea and Dove. Both are great on my skin. No...","[Notes_Cleansing, Notes_Convenience, Notes_Pro..."


In [ ]:
from flask import Flask, send_file

excel_file_path = 'Task_Output_data.xlsx'
final_test_df1.to_excel(excel_file_path, index=False)

